In [15]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from models import Conv2NetRegressor
from keras import backend as K
K.set_image_data_format('channels_last')

In [16]:
train_dataset = BatchGenerator(box_output_format=['class_id'])
validation_dataset = BatchGenerator(box_output_format=['class_id'])

train_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_train_split_391087-70-10-2000.csv', 
                        images_dir='../../ssd_keras/dataset/',
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'])

validation_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_train_split_391087-70-10-2000.csv', 
                             images_dir='../../ssd_keras/dataset/',
                             input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'])

In [17]:
img_height, img_width, img_depth = (64,64,3)

epochs = 100

train_batch_size = 64
shuffle = True
ssd_train = False

validation_batch_size = 32


In [18]:
train_generator = train_dataset.generate(batch_size=train_batch_size,
                                         shuffle=shuffle,
                                         ssd_train=ssd_train,
                                         returns={'processed_labels'},
                                         resize=(img_height, img_width))

validation_generator = validation_dataset.generate(batch_size=validation_batch_size,
                                                   shuffle=shuffle,
                                                   ssd_train=ssd_train,
                                                   returns={'processed_labels'},
                                                   resize=(img_height, img_width))

print("Number of images in the dataset:", train_dataset.get_n_samples())
print("Number of images in the dataset:", validation_dataset.get_n_samples())

Number of images in the dataset: 273761
Number of images in the dataset: 273761


In [19]:
steps_per_epoch = train_dataset.get_n_samples()/train_batch_size
validation_steps = validation_dataset.get_n_samples()/validation_batch_size

In [20]:
conv2net = Conv2NetRegressor(img_width, img_height, img_depth)

In [21]:
conv2net.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               6553800   
_________________________________________________________________
activation_5 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
__________

In [22]:
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00001, amsgrad=True)

In [23]:
checkpoint = ModelCheckpoint(filepath='callbacks/conv2net/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=True,
                           period=5)
callbacks = [checkpoint]

In [27]:
conv2net.model.compile(loss='mean_squared_error', optimizer=optimizer)

In [28]:
conv2net.model.fit_generator(train_generator, epochs=epochs, 
                             steps_per_epoch=steps_per_epoch, 
                             validation_data=validation_generator,
                             validation_steps=validation_steps,
                             callbacks=callbacks)

Epoch 1/100
  41/4277 [..............................] - ETA: 2:24:05 - loss: 109457553.5430

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/keras/engine/training.py", line 2192, in fit_generator
    generator_output = next(output_generator)
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/data_utils.py", line 787, in get
    time.sleep(self.wait_time)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-30910989c9bf>", line 5, in <module>
    callbacks=callbacks)
  File "/usr/local/lib/python3.5/dist-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.5/dist-packages/keras/models.py", line 1276, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.5/dist-packages/keras/legacy/interfaces

TypeError: Can't convert 'list' object to str implicitly